In [1]:
import torch, torchvision
from pathlib import Path

model = torchvision.models.mobilenet_v2(
    weights="MobileNet_V2_Weights.DEFAULT"
)
model.eval()          
print("✅ MobileNet V2 chargé.")


0.9%

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\Ali Abbas/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth


100.0%


✅ MobileNet V2 chargé.


In [2]:
dummy = torch.zeros((1, 3, 224, 224))       
onnx_path = Path("mobilenetv2.onnx")

torch.onnx.export(
    model, dummy, onnx_path,
    input_names=["input"], output_names=["output"],
    dynamic_axes={"input": {0: "batch"}, "output": {0: "batch"}},
    opset_version=12
)
print("✅ ONNX exporté :", onnx_path.resolve())


✅ ONNX exporté : C:\Users\Ali Abbas\Desktop\IA_web\mobilenetv2.onnx


In [3]:
from torchvision.models import MobileNet_V2_Weights
import json

labels = MobileNet_V2_Weights.DEFAULT.meta["categories"]
with open("labels.json", "w", encoding="utf-8") as f:
    json.dump(labels, f, indent=2, ensure_ascii=False)

print("✅ labels.json écrit (", len(labels), "classes).")


✅ labels.json écrit ( 1000 classes).


In [ ]:
import onnxruntime as ort
import numpy as np

sess = ort.InferenceSession("mobilenetv2.onnx",
                            providers=["CPUExecutionProvider"])

dummy_np = np.random.rand(1, 3, 224, 224).astype(np.float32)
outputs  = sess.run(None, {"input": dummy_np})

print("ONNX output shape :", outputs[0].shape) 


ONNX output shape : (1, 1000)


In [5]:
import json, numpy as np

labels = json.load(open("labels.json", encoding="utf-8"))
scores = outputs[0][0]
top5   = scores.argsort()[-5:][::-1]

print("Top-5 sur entrée aléatoire (juste pour valider le mapping) :")
for idx in top5:
    print(f"{labels[idx]}  —  score {scores[idx]:.4f}")


Top-5 sur entrée aléatoire (juste pour valider le mapping) :
kite  —  score 4.0398
bee eater  —  score 3.7040
white stork  —  score 3.5105
bald eagle  —  score 3.1918
parachute  —  score 3.1535
